In [3]:
# YoutubeModule Class
from CrawlingModule import YoutubeBuilder

# KEY 파일로 읽어오기
KEY_FILE = open('./API_KEY.txt', 'r')

# API_KEY 할당
API_KEY = KEY_FILE.readline().split(':')[1] 

# YoutbeModule Instarnce 
bulider = YoutubeBuilder(API_KEY)

In [2]:
channelId = bulider.search_get_channelId('침착맨')
video_id_list = bulider.search_get_videoId_in_channel(channelId=channelId)
video_list = bulider.get_videos_in_videoId_list(videoId_list=video_id_list)

In [4]:
comments_list = bulider.get_comments(video_id_list)

In [1]:
# import pandas as pd

# # video_id,category_id,category_name,title,views_count,like_count,uploaded_at
# chim_df = pd.DataFrame(video_list, columns=['video_id', 'category_id', 'category_name', 'title', 'views_count', 'like_count', 'uploaded_at', 'tags'])

# # chim_df.created_at = pd.to_datetime(chim_df.uploaded_at)
# # df.to_csv('./데이터/침착맨_videos.csv', index=False)

# chim_df

In [1]:
# import pandas as pd
# import Preprocessing as prep
# chim_df = pd.read_csv('./데이터/침착맨_videos.csv', encoding='utf-8', engine='python')
# chim_df.uploaded_at = pd.to_datetime(chim_df.uploaded_at)


# chim_df.sort_values(by='views_count', ascending=False).head(10)
# chim_df.sort_values(by='like_count', ascending=False).head(10)

# chim_df.description = chim_df.description.apply(lambda x : prep.extract_hashtags(x, name='#침착맨 '))
# chim_df.description = chim_df.description.apply(lambda x : prep.remove_other_hashtag(x, chim_df, './chim_dict.txt'))

In [23]:
import pandas as pd
import numpy as np
import Preprocessing as prep
from konlpy.tag import Twitter
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from PIL import Image
from collections import Counter

chim_comment_df = pd.read_csv('./데이터/chim_comments.csv', encoding='utf-8', engine='python')
chim_comment_df

,video_id,comment_id,comment,like_count,created_at,updated_at
0,7QnVBNbTVeM,Ugzmf2KCLNfGMvjOAIx4AaABAg,침소리와. 궤소리의 만남,0,2023-02-27T06:21:37Z,2023-02-27T06:21:37Z
1,7QnVBNbTVeM,UgwcByZZhnHLQtFb_od4AaABAg,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 이날따라 침방장 왜케 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,0,2023-02-27T06:19:49Z,2023-02-27T06:19:49Z
2,7QnVBNbTVeM,UgzK43w3es4_sUES3-p4AaABAg,"오늘 영상을 요약하자면 ""안드로이드는 전기양의 꿈을 꾸는가?""입니다 아아 필립K딕 ...",0,2023-02-27T06:01:43Z,2023-02-27T06:01:43Z
3,7QnVBNbTVeM,UgwTAaUHneZhtpK_LVZ4AaABAg,궤도님 기계가 분명합니다,0,2023-02-27T05:55:00Z,2023-02-27T05:55:00Z
4,7QnVBNbTVeM,Ugxs2qGT1LRd1z0gb9Z4AaABAg,월드컵 특강 너무 좋은데?,0,2023-02-27T05:53:29Z,2023-02-27T05:53:29Z
...,...,...,...,...,...,...
346128,SRS4AeJzhoc,UghVMiKNyVyI6HgCoAEC,아재 게임피지컬 ㅁㅊㄷ ㅁㅊ 답답,0,2016-05-31T10:57:36Z,2016-05-31T10:57:36Z
346129,SRS4AeJzhoc,Ugg8ahjVX6z-13gCoAEC,ㅋㅋㅋㅋㅋㅋ 초반부 설명하는거 웰케 웃겨요,0,2016-05-31T10:39:53Z,2016-05-31T10:39:53Z
346130,SRS4AeJzhoc,UgjYCSAFu6wUQ3gCoAEC,침하,0,2016-05-31T10:39:45Z,2016-05-31T10:39:45Z
346131,SRS4AeJzhoc,UggPw6PSvwpGGngCoAEC,캬 갓겜을 이제서야,0,2016-05-31T10:36:26Z,2016-05-31T10:36:26Z


In [24]:
video_comment_list = list()
stopwords = prep.stopwords('./데이터/stopwords.txt')
unique_video_id = chim_comment_df.video_id.unique()

video_id_and_count = dict()

for video_id in unique_video_id:
    video_id_and_count[video_id] = chim_comment_df.loc[chim_comment_df.video_id == video_id].like_count.sum()

In [53]:
# 비디오별 댓글 좋아요 수
comments_like_sum = sorted(video_id_and_count.items(), key=lambda x:x[1], reverse=True)

video_on_comments_like_sum = list()

for idx, value in enumerate(comments_like_sum):
    if idx == 10:
        break
    video_on_comments_like_sum.append(value)

video_on_comments_like_sum

[('9LQsLPsKRq8', 249220),
 ('53XqM_ofaKw', 190285),
 ('IfOSBmBCWw0', 179199),
 ('Z1vgOoIuP5M', 177677),
 ('pkXiOnXAO90', 159250),
 ('IxaOybVE50Y', 157263),
 ('hnanNlDbsE4', 156647),
 ('CEDzUn9pOG0', 152166),
 ('WeGDixN-u48', 138191),
 ('V1ryDPaC6mo', 124719)]

In [26]:
twitter = Twitter()
test_list = []
new_df = list()

for video_id in video_on_comments_like_sum:
    new_df.append(pd.DataFrame(chim_comment_df.loc[chim_comment_df.video_id == video_id[0]].comment.apply(twitter.nouns)))
    


In [56]:
count = 0
text_list = list()
string = ''
font_path = r'C:/Windows/Fonts/malgun.ttf'
youtube_mask = np.array(Image.open('./데이터/youtube_logo.jfif'))
new_df = list()
for i in range(10):
    new_df = pd.read_csv(f'./데이터/test{i}.csv')
    print(new_df)


# for comment in new_df.comment.values:
#     string += comment
    
# text_list.append(string)

# wc = WordCloud(font_path=font_path, background_color='white', stopwords=['개소리', '존나', '개'], mask=youtube_mask,
#        max_font_size=30, scale=7).generate(text_list[0])
# plt.figure(figsize=(10, 5))
# plt.axis('off')
# plt.imshow(wc, interpolation='bilinear')
# plt.show()
# wc.to_file(filename="침착맨_원피스.png")


                                                comment
0     ['관', '매직', '황건적', '당', '의', '난', '유비', '등장', ...
1                                                ['진짜']
2                                                ['자고']
3                             ['병건', '역사', '선생님', '한번']
4                                                 ['끝']
...                                                 ...
8993                      ['연휴', '이', '연차', '크루', '삥뽕']
8994             ['연휴', '마무리', '침촉맨', '삼국지', '이건', '꽤']
8995                                  ['침착', '맨', '계획']
8996                            ['걸', '마지막', '날', '편성']
8997                                              ['간']

[8998 rows x 1 columns]
                                             comment
0     ['쏙쏙', '수면', '도움', '저', '또', '기억', '안나', '서요']
1                                ['이제', '원피스', '요약']
2                                                 []
3                                       ['개인', '저장']
4                          ['동아시아', '급

In [10]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

from PIL import Image


font_path = r'C:/Windows/Fonts/malgun.ttf'

youtube_mask = np.array(Image.open('./데이터/youtube_logo.jfif'))

wc = WordCloud(font_path=font_path, background_color='white', stopwords=['개소리', '존나', '개'], mask=youtube_mask,
               max_font_size=30, scale=7, colors=['red', '']).generate(text_list[0])
plt.figure(figsize=(10, 5))
plt.axis('off')
plt.imshow(wc, interpolation='bilinear')
plt.show()
# wc.to_file(filename="침착맨_복숭아_대전.png")

TypeError: __init__() got an unexpected keyword argument 'colr'